In [1]:
import numpy as np, pandas as pd, scipy.stats as st
from scipy.stats import t, norm
from joblib import Parallel, delayed

In [2]:
data = pd.read_csv('data.csv')

num_obvs = 100_000
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
sector_indices = data['sector'].values
sec_loading = r[sector_indices]
datat = norm.ppf(data['p'])

In [3]:
def process_obs(obs):
    m_factor = obs[0]  
    sec_factor = obs[:len(r)][sector_indices]  
    res_factor = obs[len(r):]  

    control_variate = (
            r[0]**0.5 * m_factor
            + (sec_loading - r[0])**0.5 * sec_factor
            + (1 - sec_loading)**0.5 * res_factor
    )

    ind = control_variate < datat
    loss = np.zeros(len(data))

    if np.any(ind):
        loss[ind] = data.loc[ind, 'm'].values + data.loc[ind, 'd'].values * np.clip(t.rvs(df=3, size=sum(ind)), -5, 5)


    # expected_control = np.mean(control_variate)
    # alpha = np.cov(loss, control_variate)[0,1] / np.var(control_variate)
    # loss -= alpha * (control_variate - expected_control)

    return np.sum(loss), np.var(loss)

In [6]:
factors = np.random.normal(0, 1, (num_obvs, len(r) + len(data)))
antithetic_factors = -factors
combined_factors = np.vstack([factors, antithetic_factors])

answers = Parallel(n_jobs=-1, verbose=5)(delayed(process_obs)(obs) for obs in combined_factors)
answers = np.array(answers)

sample_losses = answers[:, 0]
sample_vars = answers[:, 1]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 1616 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 8208 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 18576 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 31248 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done 46224 tasks      | elapsed:   23.0s
[Parallel(n_jobs=-1)]: Done 63504 tasks      | elapsed:   28.3s
[Parallel(n_jobs=-1)]: Done 83088 tasks      | elapsed:   35.6s
[Parallel(n_jobs=-1)]: Done 104976 tasks      | elapsed:   43.4s
[Parallel(n_jobs=-1)]: Done 129168 tasks      | elapsed:   52.8s
[Parallel(n_jobs=-1)]: Done 155664 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 184464 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 200000 out of 200000 | elapsed:  1.4min finished


In [7]:
VaR_antithetic = np.percentile(-np.array(sample_losses), 99.9)

In [9]:
print("Średnia wariancji strat:", np.mean(sample_vars))
print("VaR antetyczny (100-ty wynik):", VaR_antithetic)

Średnia wariancji strat: 1334.6152949979016
VaR antetyczny (100-ty wynik): 23310.16054431194
